<a href="https://colab.research.google.com/github/piushag/591NR/blob/main/Adversarial_Attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
def move_image_right(image, num_pixels):
    # Pad the left side of the image with zeros
    pad = torch.zeros((3, num_pixels, 32))
    padded_image = torch.cat((pad, image), dim=1)
    # Crop the right side of the image to remove the padded pixels
    cropped_image = padded_image[:, :32, :]
    # Return the moved image
    return cropped_image

def move_image_left(image, num_pixels):
    # Get the width of the image
    width = image.shape[2]
    # Create a tensor of zeros with the same dimensions as the original image
    moved_image = torch.zeros_like(image)
    # Copy the pixels from the original image to the moved image with a leftward shift
    moved_image[:, :, :width-num_pixels] = image[:, :, num_pixels:]
    # Return the moved image
    return moved_image

def move_image_up(image, num_pixels):
    # Get the height of the image
    height = image.shape[1]
    # Create a tensor of zeros with the same dimensions as the original image
    moved_image = torch.zeros_like(image)
    # Copy the pixels from the original image to the moved image with an upward shift
    moved_image[:, :height-num_pixels, :] = image[:, num_pixels:, :]
    # Return the moved image
    return moved_image


def get_right_diff_channel(image):
    right_image = move_image_right(image,3)
    return torch.mean((image-right_image), dim=0, keepdim=True)

def get_left_diff_channel(image):
    left_image = move_image_left(image,3)
    return torch.mean((image-left_image), dim=0, keepdim=True)

def get_up_diff_channel(image):
    up_image = move_image_up(image,3)
    return torch.mean((image-up_image), dim=0, keepdim=True)

class AddNewChannels:
    """Add a fourth channel to the CIFAR-10 images with all zero values."""
    def __call__(self, img):
        return torch.cat((img, get_right_diff_channel(img), get_left_diff_channel(img), get_up_diff_channel(img)), dim=0)


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    AddNewChannels(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
])

# Define the transformations to be applied to the data
transform_test = transforms.Compose([
    transforms.ToTensor(),
    AddNewChannels(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
])

# Load the CIFAR-10 dataset
trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Load a pre-trained SqueezeNet model
net = torchvision.models.squeezenet1_1()

# Modify the first convolutional layer to accept 6 channels
net.features[0] = torch.nn.Conv2d(6, 64, kernel_size=3, stride=2)

# Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the pre-trained SqueezeNet on the 10000 test images: %d %%' % (100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified
[1,   100] loss: 3.844
[1,   200] loss: 3.452
[1,   300] loss: 3.255
[2,   100] loss: 3.225
[2,   200] loss: 3.135
[2,   300] loss: 3.104
[3,   100] loss: 3.029
[3,   200] loss: 2.927
[3,   300] loss: 3.012
[4,   100] loss: 2.961
[4,   200] loss: 2.887
[4,   300] loss: 2.851
[5,   100] loss: 2.831
[5,   200] loss: 2.822
[5,   300] loss: 2.783
[6,   100] loss: 2.789
[6,   200] loss: 2.732
[6,   300] loss: 2.728
[7,   100] loss: 2.729
[7,   200] loss: 2.681
[7,   300] loss: 2.711
[8,   100] loss: 2.638
[8,   200] loss: 2.640
[8,   300] loss: 2.622
[9,   100] loss: 2.612
[9,   200] loss: 2.586
[9,   300] loss: 2.598
[10,   100] loss: 2.558
[10,   200] loss: 2.626
[10,   300] loss: 2.561
Finished Training
Accuracy of the pre-trained SqueezeNet on the 10000 test images: 50 %


In [ ]:
# Define the adversarial attack
def fgsm_attack(image, epsilon, data_grad):
    # Calculate the sign of the gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adding the signed gradient to the original image
    perturbed_image = image + epsilon * sign_data_grad
    # Clip the perturbed image so that its pixel values are between 0 and 1
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [ ]:
import torch.nn.functional as F

# Generate adversarial examples for the test dataset using FGSM attack
epsilon = 0.1
correct = 0
total = 0
for data in testloader:
    # Get the input image and label
    images, labels = data
    # Set requires_grad attribute of tensor to True, since we want to compute the gradient with respect to the input image
    images.requires_grad = True
    # Forward pass the input image through the model
    outputs = net(images)
    # Calculate the loss
    loss = F.cross_entropy(outputs, labels)
    # Zero out the gradients
    net.zero_grad()
    # Backward pass to compute the gradient
    loss.backward()
    # Get the gradient of the loss with respect to the input image
    data_grad = images.grad.data
    # Call the adversarial attack function to generate the perturbed image
    perturbed_image = fgsm_attack(images, epsilon, data_grad)
    outputs = net(perturbed_image)
    # Get the predicted label for the perturbed image
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Adversarial accuracy of the pre-trained SqueezeNet on the 10000 test images: %d %%' % (100 * correct / total))

Adversarial accuracy of the pre-trained SqueezeNet on the 10000 test images: 14 %
